In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold, cross_val_score
from xgboost import XGBRegressor  
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.metrics import r2_score
import optuna
from xgboost import XGBRegressor

In [3]:
df= pd.read_csv(r"D:\Intellipaat\kegal datasets\april -s4e4\train.csv")
df.drop(columns=['id'],inplace = True)
df

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9
...,...,...,...,...,...,...,...,...,...
90610,M,0.335,0.235,0.075,0.1585,0.0685,0.0370,0.0450,6
90611,M,0.555,0.425,0.150,0.8790,0.3865,0.1815,0.2400,9
90612,I,0.435,0.330,0.095,0.3215,0.1510,0.0785,0.0815,6
90613,I,0.345,0.270,0.075,0.2000,0.0980,0.0490,0.0700,6


In [4]:
path_to_input_folder = "D:\\Intellipaat\\kegal datasets\\april -s4e4\\abalone\\"
column_names = ['Sex','Length','Diameter','Height','Whole weight','Whole weight.1','Whole weight.2','Shell weight','Rings']
org_df = pd.read_csv(path_to_input_folder + "abalone.data", names=column_names)

In [5]:
data = pd.concat([df,org_df])
data

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [6]:
data.shape

(94792, 9)

In [7]:
encoder = LabelEncoder()

In [8]:
data['Sex'] = encoder.fit_transform(data['Sex'].values.reshape(-1,1))

C:\Users\mahes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
data

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,0,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,1,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,2,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,1,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9
...,...,...,...,...,...,...,...,...,...
4172,0,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,2,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,2,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,0,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [10]:
X = data.drop(columns=['Rings']) 
Y = data['Rings']

In [11]:
data.head()

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,0,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,1,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,2,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,1,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [15]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

# Define the best parameters obtained from the optimization
best_params = {
    'n_estimators': 391,
    'max_depth': 9,
    'num_leaves': 44,
    'learning_rate': 0.024306079023490815,
    'min_child_samples': 3,
    'min_child_weight': 9.177604531077481,
    'subsample': 0.6414373404614598,
    'colsample_bylevel': 0.5580026903871661,
    'reg_alpha': 0.03373897187756375,
    'reg_lambda': 0.028360521678445487
}

# Create an XGBoost regressor with the best parameters
best_xgbr = XGBRegressor(random_state=0, **best_params)

# Create a pipeline with standard scaler and XGBoost regressor
best_pipe = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', best_xgbr)
    ]
)

# Fit the pipeline to the entire dataset
best_pipe.fit(X, Y)

# Make predictions for 'Rings'
predictions = best_pipe.predict(X)

# Now you have predictions for the 'Rings' column using the optimized XGBoost regressor with the best parameters


[15:25:52] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "min_child_samples", "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




In [16]:
best_params

{'n_estimators': 391,
 'max_depth': 9,
 'num_leaves': 44,
 'learning_rate': 0.024306079023490815,
 'min_child_samples': 3,
 'min_child_weight': 9.177604531077481,
 'subsample': 0.6414373404614598,
 'colsample_bylevel': 0.5580026903871661,
 'reg_alpha': 0.03373897187756375,
 'reg_lambda': 0.028360521678445487}

In [18]:
best_pipe

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                              colsample_bylevel=0.5580026903871661,
                              colsample_bynode=1, colsample_bytree=1,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.024306079023490815, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=9, max_leaves=0, min_child_samples=3,
                              min_child_weight=9.177604531077481, missing=nan,
                              monotone_constraints='()', n_estimators=391,
                              n_jobs=0, num_leaves=44, num_parallel_tree=1,
                              predictor='auto', random_state=0, ...))])

In [19]:
predictions

array([10.235812 , 10.611419 ,  4.051821 , ..., 10.9556265,  9.077377 ,
       11.673474 ], dtype=float32)

In [20]:
test_data= pd.read_csv(r"D:\Intellipaat\kegal datasets\april -s4e4\test.csv")
test_data

,id,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight
0,90615,M,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005
1,90616,M,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750
2,90617,M,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405
3,90618,M,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350
4,90619,I,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050
...,...,...,...,...,...,...,...,...,...
60406,151021,I,0.345,0.260,0.085,0.1775,0.0735,0.0265,0.0500
60407,151022,F,0.525,0.410,0.145,0.8445,0.3885,0.1670,0.2050
60408,151023,I,0.590,0.440,0.155,1.1220,0.3930,0.2000,0.2650
60409,151024,F,0.660,0.525,0.190,1.4935,0.5885,0.3575,0.4350


In [21]:
test = test_data.drop(columns=['id'])
test

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight
0,M,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005
1,M,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750
2,M,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405
3,M,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350
4,I,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050
...,...,...,...,...,...,...,...,...
60406,I,0.345,0.260,0.085,0.1775,0.0735,0.0265,0.0500
60407,F,0.525,0.410,0.145,0.8445,0.3885,0.1670,0.2050
60408,I,0.590,0.440,0.155,1.1220,0.3930,0.2000,0.2650
60409,F,0.660,0.525,0.190,1.4935,0.5885,0.3575,0.4350


In [22]:
test['Sex'] = encoder.fit_transform(test_data['Sex'].values.reshape(-1,1))

C:\Users\mahes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
test

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight
0,2,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005
1,2,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750
2,2,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405
3,2,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350
4,1,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050
...,...,...,...,...,...,...,...,...
60406,1,0.345,0.260,0.085,0.1775,0.0735,0.0265,0.0500
60407,0,0.525,0.410,0.145,0.8445,0.3885,0.1670,0.2050
60408,1,0.590,0.440,0.155,1.1220,0.3930,0.2000,0.2650
60409,0,0.660,0.525,0.190,1.4935,0.5885,0.3575,0.4350


In [24]:
predictions = best_pipe.predict(test)
predictions

array([ 9.979823 ,  9.756761 , 10.139375 , ..., 12.639607 , 13.5279045,
        8.670358 ], dtype=float32)

In [25]:
# file = np.round(predictions).astype(int)
# file

array([10, 10, 10, ..., 12, 13,  9])

In [25]:
submission_df = pd.DataFrame({'id': test_data['id'], 'Rings' : predictions})
submission_df

,id,Rings
0,90615,9.979823
1,90616,9.756761
2,90617,10.139375
3,90618,10.490775
4,90619,7.612429
...,...,...
60406,151021,6.385515
60407,151022,9.331569
60408,151023,12.639607
60409,151024,13.527905


In [26]:
submission_df.to_csv('submission_xgb_param.csv', index=False)